In [ ]:
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup

import time

import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 경향신문 2024년 1월 1일부터 12월 31일 사이 '사다리'라는 키워드로 검색한 기사 수집하기
# 이 작업을 수행하기 전에 링크 전체 주소에 페이지 표시가 포함되어있는지 확인 필요

FIRST_PAGE = 'https://search.khan.co.kr/?q=%EC%82%AC%EB%8B%A4%EB%A6%AC&media=khan&page=1&section=0&term=5&startDate=2024-01-01&endDate=2024-12-31&sort=1'

# TARGET_URL 에서는 페이지 번호가 1부터 반복해서 입력될 수 있도록 제거(페이지 번호 위치가 어디인지 위, 아래 비교 필요)
TARGET_URL_KEYWORD = 'https://search.khan.co.kr/?q=%EC%82%AC%EB%8B%A4%EB%A6%AC&media=khan&page=' # 메인 + 키워드
TARGET_URL_REST = '&section=0&term=5&startDate=2024-01-01&endDate=2024-12-31&sort=1' # 페이지 및 날짜
TARGET_URL = TARGET_URL_KEYWORD + TARGET_URL_REST

print(TARGET_URL)

https://search.khan.co.kr/?q=%EC%82%AC%EB%8B%A4%EB%A6%AC&media=khan&page=&section=0&term=5&startDate=2024-01-01&endDate=2024-12-31&sort=1


In [ ]:
TITLE_OF_ARTICLE = [] # 기사 제목을 저장할 리스트
URL_OF_ARTICLE = []   # 기사 URL을 저장할 리스트

# 사용자 에이전트 설정: 웹 서버가 요청을 실제 사용자의 브라우저 요청으로 인식하도록 설정
url = urllib.request.Request(FIRST_PAGE,
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})

# 웹 페이지 열기 및 HTML 가져오기
html = urllib.request.urlopen(url) # 설정된 요청으로 웹 페이지 열기
soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8') # 가져온 HTML을 BeautifulSoup 객체로 변환

# 웹 페이지에서 필요한 정보 추출 및 저장
for title in soup.select('ul.list > li article > div > a'): # 기사 제목 포함 태그 선택
    TITLE_OF_ARTICLE.append(title.get_text()) # 링크 텍스트(기사 제목) 추출 및 저장. HTML 태그 안에 있는 순수한 텍스트만 꺼냄

    article_URL = title['href'] # 기사 URL(href 속성) 추출 및 저장
    URL_OF_ARTICLE.append(article_URL)

# 결과 출력
print("\n".join(TITLE_OF_ARTICLE))
print("\n".join(URL_OF_ARTICLE))

오세훈 시장 신년사 “규제철폐와 첨단산업 육성으로 위기 극복”
‘전북 대표 중소기업’ 공모로 12개사 인증
전북도, ‘유망중소기업’ 12곳 성장 돕는다
스프링클러 설치 확대·완강기 기준 강화 등 숙박시설 소방안전 대책 마련
끊긴 계층이동 사다리···소득 상위 20% 진입률 ‘3.5%’
‘오겜 2’ 황동혁 “이런 시국에 시즌2 공개 마음 무거워···위로와 도움이 되길”
뿌리 깊은 중독성과 돈맛…“성인이 된 지금도 스스로와 싸웁니다”
한때 ‘도박 중독 청소년’ 정문·주민은 지금도 극복 안간힘…“돈 제대로 가르쳐야”
[기자메모] 청년에게 코인 투자하라는 여당 대표 한동훈
이재명 ‘위증교사 1심’ 선고 날···경찰 늘고, 법원 입구에도 질서유지선
https://www.khan.co.kr/article/202412311127001
https://www.khan.co.kr/article/202412252013015
https://www.khan.co.kr/article/202412251104001
https://www.khan.co.kr/article/202412191145011
https://www.khan.co.kr/article/202412181608001
https://www.khan.co.kr/article/202412091526001
https://www.khan.co.kr/article/202412030600005
https://www.khan.co.kr/article/202412021732011
https://www.khan.co.kr/article/202411280600011
https://www.khan.co.kr/article/202411251200041


In [ ]:
# 뉴스 제목으로부터 링크를 가져오는 함수 정의

def get_link_from_news_title(page_num, URL):
    for i in range(page_num):   # 지정한 페이지 수만큼 반복
        current_page_num = 1 + i    # 현재 페이지 번호 계산 (1부터 시작)
        position = URL.index('page=')   # 'page=' 문자열의 위치를 찾음
        URL_with_page_num = URL[:position+5] + str(current_page_num) + URL[position+5:]  # 해당 위치에 현재 페이지 번호 삽입

        # HTTP 요청 객체 생성 (크롤링 방지 우회용 User-Agent 설정
        URL_with_page_num = urllib.request.Request(URL_with_page_num,
                                                   headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})

        source_code_form_url = urllib.request.urlopen(URL_with_page_num)  # URL에 요청을 보내고 HTML 응답 수신
        soup = BeautifulSoup(source_code_form_url, 'html.parser', from_encoding='utf-8')  # BeautifulSoup으로 HTML 파싱

        for title in soup.select('ul.list > li article > div > a'):
            TITLE_OF_ARTICLE.append(title.get_text())
            article_URL = title['href']   # <a> 태그의 href 속성값 (기사 링크) 추출
            URL_OF_ARTICLE.append(article_URL)   # 기사 링크를 리스트에 추가

        if (i) % 10 == 0:   # 매 10번째 루프마다
            time.sleep(2)   # 서버 과부하 방지를 위한 2초 대기
            print(f'{(i) * 10} articles complete')     # 진행 상황 출력

In [ ]:
# 전체 기사 수를 받아 필요한 페이지 수를 계산하는 함수 정의
# 아래 코드는 내가 보고있는 페이지의 게시물 개수에 따라 숫자가 달라지므로 반드시 확인

def get_total_num_of_article(number):
    if(number % 10 >= 1):   # 10으로 나눈 나머지가 1 이상이면 (즉, 나누어떨어지지 않으면)
        page_num = int((number / 10) + 1)   # 몫에 1을 더하여 추가 페이지 확보 (반올림 효과)
    else:   # 10으로 나누어떨어지는 경우
        page_num = int(number / 10)     # 정확히 나눈 몫이 곧 페이지 수
    return page_num

page_num = get_total_num_of_article(170)
page_num

17

In [ ]:
# 기사 제목, URL 을 저장할 리스트 생성
TITLE_OF_ARTICLE = []
URL_OF_ARTICLE = []

# get_link_from_news_title 함수를 호출하여 지정된 페이지 수와 URL을 기반으로 뉴스 기사의 제목, URL 수집
get_link_from_news_title(17, TARGET_URL)

0nd article complete
100nd article complete


In [ ]:
# 판다스 DataFrame 생성

df = pd.DataFrame(list(zip(URL_OF_ARTICLE, TITLE_OF_ARTICLE)),      # 두 리스트를 각각 (URL, 제목) 튜플로 묶은 후 데이터프레임 처리
                  columns=['Url', 'Title']      # 컬럼명을 'Url'과 'Title'로 지정
)

print(len(df))  # 생성된 DataFrame의 행 개수 출력 (=기사 수)
df  # 전체 DataFrame 출력

170


,Url,Title
0,https://www.khan.co.kr/article/202412311127001,오세훈 시장 신년사 “규제철폐와 첨단산업 육성으로 위기 극복”
1,https://www.khan.co.kr/article/202412252013015,‘전북 대표 중소기업’ 공모로 12개사 인증
2,https://www.khan.co.kr/article/202412251104001,"전북도, ‘유망중소기업’ 12곳 성장 돕는다"
3,https://www.khan.co.kr/article/202412191145011,스프링클러 설치 확대·완강기 기준 강화 등 숙박시설 소방안전 대책 마련
4,https://www.khan.co.kr/article/202412181608001,끊긴 계층이동 사다리···소득 상위 20% 진입률 ‘3.5%’
...,...,...
165,https://www.khan.co.kr/article/202401102000005,[경향의 눈] 전두환의 교육개혁
166,https://www.khan.co.kr/article/202401101705001,"빌라 살리기, 실수요자에게 도움될까···거주여건 나빠질 수도[1·10 주택대책]"
167,https://www.khan.co.kr/article/202401101349001,최근 5년간 사다리 작업 중 추락사한 노동자 200명 넘어
168,https://www.khan.co.kr/article/202401082016005,[기자 칼럼] 쓸모없는 아이들


In [ ]:
# DataFrame을 CSV 파일로 저장
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/KyungH.csv",     # 저장할 파일 경로 (Google Drive 내 지정 경로)
    index=False,    # 행 번호(인덱스)를 저장하지 않음
    encoding='utf-8-sig'    # 인코딩 방식: 한글 깨짐 방지를 위해 'utf-8-sig' 사용
)

## Collect texts from the article

In [ ]:
# 위 작업에서 바로 이어서 진행할 경우 필요없음

df = pd.read_csv("/content/drive/MyDrive/CLASS/202502_NLP/KyungH.csv")
df

,Url,Title
0,https://www.khan.co.kr/article/202412311127001,오세훈 시장 신년사 “규제철폐와 첨단산업 육성으로 위기 극복”
1,https://www.khan.co.kr/article/202412252013015,‘전북 대표 중소기업’ 공모로 12개사 인증
2,https://www.khan.co.kr/article/202412251104001,"전북도, ‘유망중소기업’ 12곳 성장 돕는다"
3,https://www.khan.co.kr/article/202412191145011,스프링클러 설치 확대·완강기 기준 강화 등 숙박시설 소방안전 대책 마련
4,https://www.khan.co.kr/article/202412181608001,끊긴 계층이동 사다리···소득 상위 20% 진입률 ‘3.5%’
...,...,...
165,https://www.khan.co.kr/article/202401102000005,[경향의 눈] 전두환의 교육개혁
166,https://www.khan.co.kr/article/202401101705001,"빌라 살리기, 실수요자에게 도움될까···거주여건 나빠질 수도[1·10 주택대책]"
167,https://www.khan.co.kr/article/202401101349001,최근 5년간 사다리 작업 중 추락사한 노동자 200명 넘어
168,https://www.khan.co.kr/article/202401082016005,[기자 칼럼] 쓸모없는 아이들


In [ ]:
# 기사 날짜와 본문 수집 시도

# 주어진 URL에 대한 요청 객체 생성, 여기서 사용자 에이전트를 설정하여 브라우저에서의 요청처럼 만듦
url = urllib.request.Request(df['Url'][1],
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})

source_code_from_url = urllib.request.urlopen(url) # 설정한 요청으로부터 소스 코드를 가져옴
soup = BeautifulSoup(source_code_from_url, 'html.parser', from_encoding='utf-8') # 가져온 소스 코드를 BeautifulSoup을 사용하여 파싱, 이때 HTML의 구조를 분석하기 위해 'html.parser' 사용
content = soup.select('div.art_body > p') # 기사 본문이 포함된 div 태그의 클래스 'art_body'에서 p 태그를 모두 선택하여 기사 내용 추출

# 만약 기본 구조에서 본문을 못 찾은 경우 (빈 리스트), 대체 구조로 재시도
if content == []:
    content = soup.select('p.content_text')  # 대체 구조 선택
    CONTENT = []                             # 기사 문장 저장할 리스트
    for item in content:
        CONTENT.append(item.get_text())     # 각 문단 텍스트 추출하여 리스트에 추가

    CONTENT = " ".join(CONTENT)             # 리스트의 문단들을 하나의 문자열로 합침
    CONTENT                                  # 최종 기사 본문 변수

else:
    CONTENT = []                             # 기사 문장 저장할 리스트
    for item in content:
        CONTENT.append(item.get_text())     # 각 문단 텍스트 추출하여 리스트에 추가

    CONTENT = " ".join(CONTENT)             # 리스트의 문단들을 하나의 문자열로 합침
    CONTENT                                  # 최종 기사 본문 변수

# 기사 날짜 추출 시도
try:
    date = soup.select_one('div.date > p')  # 기본 위치에서 날짜 태그 선택
    if date == None:                        # 없을 경우 대체 위치에서 추출
        date = soup.select_one('div.date > a > p').get_text()
        DATE = date[3:13]                   # 날짜 텍스트에서 실제 날짜만 추출 (문자열 슬라이싱)
        DATE
    else:
        DATE = date.get_text()[3:13]        # 기본 구조에서도 마찬가지로 슬라이싱 추출
        DATE
except:
    DATE = 'NONE'                           # 오류 발생 시 기본값 설정
    DATE

# 최종적으로 합쳐진 기사 내용을 출력
print(DATE)
print(CONTENT)

2024.12.25
전북도를 대표하는 중소기업 12개사가 선정됐다. 전북도는 전북도청에서 ‘2024년 전북도 유망중소기업 시상식’을 열고 인증서와 현판을 수여했다고 25일 밝혔다. 올해 ‘전북도 유망중소기업’ 공모에는 총 37개 기업이 지원해, 3 대 1의 높은 경쟁률을 보였다. 인증기업에는 전북지사 명의의 인증서와 인증 현판을 주며 5년 동안 전북도 경영안정자금 최대 5억원(일반기업 3억원)과 이차보전 최대 3%(일반기업 2%)까지 우대 지원한다. 또 성장 사다리 육성사업 등 지원사업에도 가점 부여 등 다양한 혜택을 제공할 방침이다. 전북도 유망중소기업 인증은 성장잠재력이 큰 중소기업을 발굴해 지역경제를 선도하는 기업으로 육성하기 위한 제도이다. 전북도는 1999년부터 도내 중소 제조기업과 지식서비스기업을 매년 10여개 선정해 우수기업으로 성장할 수 있도록 하고 있다.


In [ ]:
# 기사 URL 목록을 입력으로 받아 각 URL에서 텍스트를 추출하는 함수

def get_texts_from_news(URL):
    # URL 목록의 각 URL에 대하여 반복 실행
    for i in range(len(URL)):
        url = urllib.request.Request(URL[i], headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
        source_code_from_url = urllib.request.urlopen(url)
        soup = BeautifulSoup(source_code_from_url, 'html.parser', from_encoding='utf-8')

        content = soup.select('div.art_body > p')

        try:
            if content == []:   # 만약 기본 구조로 본문이 없을 경우
                content = soup.select('p.content_text')     # 대체 구조로 본문 재시도

                CONTENT = []    # 본문 저장용 리스트 초기화
                for item in content:    # 각 문단 반복
                    CONTENT.append(item.get_text())     # 텍스트 추출 후 리스트에 추가

                CONTENT = " ".join(CONTENT)     # 하나의 문자열로 결합
                CONTENT_OF_ARTICLE.append(CONTENT)      # 전역 본문 리스트에 추가

            else:   # 본문이 정상적으로 추출된 경우
                CONTENT = []
                for item in content:
                    CONTENT.append(item.get_text())     # 동일한 방식으로 텍스트 추출

                CONTENT = " ".join(CONTENT)
                CONTENT_OF_ARTICLE.append(CONTENT)

        except:     # 본문 추출 실패 시
            CONTENT = 'NONE'
            CONTENT_OF_ARTICLE.append(CONTENT)

        try:
            date = soup.select_one('div.date > p')      # 기본 구조에서 날짜 추출 시도
            if date == None:    # 없으면 대체 구조로 재시도
                date = soup.select_one('div.date > a > p').get_text()
                DATE = date[3:13]   # 날짜 문자열 일부 추출
                DATE_OF_ARTICLE.append(DATE)
            else:
                DATE = date.get_text()[3:13]
                DATE_OF_ARTICLE.append(DATE)

        except:     # 날짜 추출 실패 시
            DATE = 'NONE'
            DATE_OF_ARTICLE.append(DATE)


        # 서버 부하 방지 및 진행 모니터링
        if i % 5 == 0:
            time.sleep(2)    # 5개마다 2초 대기
        if i % 100 == 0:    # 100개마다 진행 상태 출력
            print(f'{i} articles complete')

In [ ]:
CONTENT_OF_ARTICLE = []
DATE_OF_ARTICLE = []

get_texts_from_news(df['Url'])

0nd article complete
100nd article complete


In [ ]:
df = pd.DataFrame(list(zip(DATE_OF_ARTICLE, df['Url'], df['Title'], CONTENT_OF_ARTICLE)), columns =['Date', 'Url', 'Title', 'Content'])

print(len(df))
df.head()

170


,Date,Url,Title,Content
0,2024.12.31,https://www.khan.co.kr/article/202412311127001,오세훈 시장 신년사 “규제철폐와 첨단산업 육성으로 위기 극복”,오세훈 서울시장이 신년사를 통해 “과감한 규제철폐로 경제와 사회의 숨통을 틔우고 활...
1,2024.12.25,https://www.khan.co.kr/article/202412252013015,‘전북 대표 중소기업’ 공모로 12개사 인증,전북도를 대표하는 중소기업 12개사가 선정됐다. 전북도는 전북도청에서 ‘2024년 ...
2,2024.12.25,https://www.khan.co.kr/article/202412251104001,"전북도, ‘유망중소기업’ 12곳 성장 돕는다",전북도를 대표하는 중소기업 12개사가 선정됐다. 전북도는 전북도청에서 ‘2024년 ...
3,2024.12.19,https://www.khan.co.kr/article/202412191145011,스프링클러 설치 확대·완강기 기준 강화 등 숙박시설 소방안전 대책 마련,스프링클러 설치 확대와 완강기 기준 강화 등 숙박시설 소방안전을 강화하기 위한 종합...
4,2024.12.18,https://www.khan.co.kr/article/202412181608001,끊긴 계층이동 사다리···소득 상위 20% 진입률 ‘3.5%’,한 해 동안 소득 계층이 상승한 국민이 10명 중 2명도 되지 않은 것으로 집계됐다...


In [ ]:
df['Content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
df['Content'].replace('', np.nan, inplace=True) # 'Content' 열에서 빈 문자열을 NaN 값으로 대체
df.dropna(subset=['Content'], inplace=True) # 'Content' 열에 NaN 값을 포함하는 모든 행을 삭제

# 처리 후 남은 행의 개수 출력
print(len(df))

170


/tmp/ipython-input-1632476055.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
/tmp/ipython-input-1632476055.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

In [ ]:
df = df[~df["Content"].str.contains(r"사다리차",     # "사다리차"라는 단어가 "Content" 컬럼에 포함된 행을 필터링
                                    case=False,     # 대소문자 구분 없이 검색
                                    na=False)]      # 결측값(NaN)은 False로 간주하여 포함되지 않도록 설정

print(len(df))
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/KyungH_final.csv",index=False, encoding='utf-8-sig')

158
